Connection with drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import IPython.display as ipd
import librosa
import librosa.display

In [ ]:
!pip install tensorflow==2.5.*

In [ ]:
!pip install llvmlite==0.31.0

!pip install resampy

# Feature extr

In [ ]:
#### Extracting MFCC's For every audio file
import pandas as pd
import os
import librosa

audio_dataset_path='/content/drive/MyDrive/project_dataset/ragam'
metadata=pd.read_csv('/content/drive/MyDrive/project_dataset/metadata/pureRaga.csv')
metadata.head(10)

,Unnamed: 0,file_name,fold,classID,class
0,0,raga-charukesi-arohanam_avarohanam.mp3,charukesi,1,charukesi
1,1,raga-charukesi-carnatic-adamodigaladha.mp3,charukesi,1,charukesi
2,2,raga-charukesi-cine-neeye_gathi_eswari.mp3,charukesi,1,charukesi
3,3,carnatic_song-adamodi_galade-charukesi-adi-thy...,charukesi,1,charukesi
4,4,raga-charukesi-signature.mp3,charukesi,1,charukesi
5,5,raga-gowri_manohari-arohanam_avarohanam.mp3,gowri_manohari,2,gowri_manohari
6,6,raga-gowri_manohari-carnatic-guruleka_etuvanti...,gowri_manohari,2,gowri_manohari
7,7,raga-gowri_manohari-cine-muthamizh_kaviye_varu...,gowri_manohari,2,gowri_manohari
8,8,raga-gowri_manohari-signature.mp3,gowri_manohari,2,gowri_manohari
9,9,raga-karaharapriya-arohanam_avarohanam.mp3,kharaharapriya,3,kharaharapriya


In [ ]:
### Check whether the dataset is imbalanced
metadata['class'].value_counts()

shankarabharanm    9
mayamalavagowla    8
charukesi          5
kharaharapriya     5
gowri_manohari     4
Name: class, dtype: int64

In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features
    

In [ ]:
import numpy as np
from tqdm import tqdm
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["fold"])+'/',str(row["file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

31it [01:53,  3.66s/it]


In [ ]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-257.26495, 57.66357, -26.706806, 18.667265, ...",charukesi
1,"[-222.41992, 74.0872, -40.357086, 13.945315, -...",charukesi
2,"[-194.89345, 150.85352, -36.30877, 36.826942, ...",charukesi
3,"[-278.32816, 79.06662, -29.88183, 21.183525, -...",charukesi
4,"[-244.1283, 61.821785, -26.80833, 22.560522, -...",charukesi


In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
X.shape


(31, 40)

In [ ]:
y.shape

(31,)

In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
### Label Encoding
###y=np.array(pd.get_dummies(y))
### Label Encoder
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [ ]:
y

In [ ]:
## Train Test Split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
X_train

In [ ]:
y

In [ ]:
X_train.shape

(24, 40)

In [ ]:
X_test.shape

(7, 40)

In [ ]:
y_train.shape

(24, 5)

In [ ]:
y_test.shape

(7, 5)

**model** **creation**

In [ ]:
#model creation


import tensorflow
# print(tf.__version__)

In [ ]:
from sklearn import metrics
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
### No of classes
num_labels=y.shape[1]

In [ ]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')


In [ ]:
## Trianing my model
from tensorflow import keras
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 1000
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
1/1 [==============================] - 0s 212ms/step - loss: 1.5348 - accuracy: 0.2917 - val_loss: 1.6392 - val_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 1.63918, saving model to saved_models/audio_classification.hdf5
Epoch 2/100
1/1 [==============================] - 0s 47ms/step - loss: 1.8237 - accuracy: 0.3333 - val_loss: 1.6393 - val_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 1.63918
Epoch 3/100
1/1 [==============================] - 0s 73ms/step - loss: 1.6948 - accuracy: 0.2500 - val_loss: 1.6394 - val_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 1.63918
Epoch 4/100
1/1 [==============================] - 0s 70ms/step - loss: 1.4668 - accuracy: 0.3750 - val_loss: 1.6395 - val_accuracy: 0.0000e+00

Epoch 00004: val_loss did not improve from 1.63918
Epoch 5/100
1/1 [==============================] - 0s 148ms/step - loss: 1.4653 - accuracy: 0.3750 - val_loss: 1.6396 - val_accuracy: 0.0000e+00

Epoch 00005: 

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.2857142984867096


In [ ]:
model.output[0]

<KerasTensor: shape=(5,) dtype=float32 (created by layer 'tf.__operators__.getitem')>

**testing**

In [ ]:
filename="/content/drive/MyDrive/project_dataset/audio/chakravaakam/Angakale.mp3"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label = (model.predict(mfccs_scaled_features) > 0.5).astype("int32")
print(predicted_label)
predicted_label = predicted_label.reshape(-1)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class

[-94.93517     77.98219     24.928661    12.809218     9.34866
  16.415455   -12.359661    15.701782    -8.868588    12.647937
  -4.6966386   12.574158   -11.557611    10.593115   -11.176537
   7.4536085  -13.286072     4.283599    -8.712832     2.1616461
  -9.041883     3.5305066   -4.531111    10.61175     -4.576577
   1.7942218  -13.299651    -3.5314746   -6.208214     1.6424209
  -6.33824     -6.4402647   -9.713605     0.40787435  -1.6858515
   0.57013285  -7.334308    -2.1905198   -5.2250013   -4.5522623 ]
[[-94.93517     77.98219     24.928661    12.809218     9.34866
   16.415455   -12.359661    15.701782    -8.868588    12.647937
   -4.6966386   12.574158   -11.557611    10.593115   -11.176537
    7.4536085  -13.286072     4.283599    -8.712832     2.1616461
   -9.041883     3.5305066   -4.531111    10.61175     -4.576577
    1.7942218  -13.299651    -3.5314746   -6.208214     1.6424209
   -6.33824     -6.4402647   -9.713605     0.40787435  -1.6858515
    0.57013285  -7.334308 

array(['chakravaakam', 'chakravaakam', 'chakravaakam', 'chakravaakam',
       'chakravaakam'], dtype='<U14')

In [ ]:
y[0:30]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [ ]:
y_test

array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
print("y_true = " + str(y_true.eval()))